In [2]:
!pip install datasets sentence-transformers faiss-cpu transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
# Import necessary libraries
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the dataset
print("Loading dataset...")
dataset = load_dataset('rotten_tomatoes', split='train[:1000]')  # Using only first 1000 samples

# Initialize the sentence transformer model
print("Initializing the sentence transformer model...")
sentence_model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# Prepare documents and their embeddings
print("Preparing document embeddings...")
documents = dataset['text']
embeddings = sentence_model.encode(documents, convert_to_numpy=True)

# Create and populate Faiss index
print("Creating Faiss index...")
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

# Save the Faiss index and document texts
print("Saving Faiss index and document texts...")
faiss.write_index(index, 'faiss_index.index')
np.save('document_texts.npy', documents)

# Load models for generation
print("Loading models...")
tokenizer = AutoTokenizer.from_pretrained('sshleifer/distilbart-cnn-6-6')
generator = AutoModelForSeq2SeqLM.from_pretrained('sshleifer/distilbart-cnn-6-6')

# Function to generate answers
def generate_answer(query):
    # Encode the query using the sentence transformer
    query_embedding = sentence_model.encode([query], convert_to_numpy=True)

    # Find similar documents
    _, I = index.search(query_embedding, 1)
    context = documents[I[0][0]]

    # Generate answer
    input_ids = tokenizer(query + " Context: " + context, return_tensors='pt')['input_ids']
    generated = generator.generate(input_ids, max_length=50)
    answer = tokenizer.decode(generated[0], skip_special_tokens=True)
    return answer, context

if __name__ == '__main__':
    # Test the model
    query = "What is the sentiment of this movie review?"
    answer, context = generate_answer(query)
    print(f"Query: {query}")
    print(f"Context: {context}")
    print(f"Answer: {answer}")

    # Another example
    query = "Summarize this movie review."
    answer, context = generate_answer(query)
    print(f"\nQuery: {query}")
    print(f"Context: {context}")
    print(f"Answer: {answer}")

Loading dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (50). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


Query: What is the sentiment of this movie review?
Context: the film is often filled with a sense of pure wonderment and excitement not often seen in today's cinema du sarcasm
Answer:  The film is often filled with a sense of pure wonderment and excitement not often seen in today's cinema du sarcasm. The film has been a hit with some of the world's most popular films. The film was released this week

Query: Summarize this movie review.
Context: this is a very fine movie -- go see it .
Answer: Summarize this movie review. Summarize: This is a very fine movie -- go see it. The movie is about a young couple who are trying to find a way to get through the tough times. The film is about
